In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf

In [50]:
data = pd.read_csv('C:/.../AutoEncoder/nyse/clean_data.csv')

In [51]:
data.head()

,date,ticker,returns,month spead,high off close,low off close,high of open,low off open,volume,Accounts Payable,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
0,12/31/2015,AAL,-0.021036,0.739998,0.005168,0.012431,0.000705,-0.016973,6788900,5.102000e+09,...,4.841500e+10,9.985000e+09,13605000000,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.000000e+00,11.39,6.681299e+08
1,12/31/2015,ABBV,0.003728,0.729999,0.011513,0.000676,0.007175,-0.005068,6019300,8.463000e+09,...,5.305000e+10,1.631400e+10,10894000000,3.945000e+09,4.910500e+10,5.305000e+10,2.285900e+10,-8.839000e+09,3.15,1.633016e+09
2,12/31/2015,ABT,-0.012099,0.599998,0.012967,0.000223,0.008571,-0.004677,4156700,5.683000e+09,...,4.124700e+10,1.415500e+10,9186000000,2.121100e+10,2.003600e+10,4.124700e+10,2.040500e+10,-1.062200e+10,2.94,1.504422e+09
3,12/31/2015,ADM,-0.000545,0.450001,0.005962,0.006310,0.007046,-0.005213,2516800,1.340700e+10,...,4.015700e+10,2.182900e+10,13505000000,1.789900e+10,2.224200e+10,4.014100e+10,6.770200e+10,0.000000e+00,2.99,6.183946e+08
4,12/31/2015,ADS,-0.043540,4.890015,0.011862,0.005709,0.015971,-0.001527,262100,1.010619e+09,...,2.242183e+10,1.625042e+10,6405559000,2.010030e+09,2.041180e+10,2.242183e+10,6.439746e+09,-3.927345e+09,8.91,6.695185e+07


In [52]:
def normalize_df(df):
        
    for col in list(df.columns):
       
        mean, std = df[col].mean(), df[col].std()

        df.loc[:, col] = (df[col] -mean) /(std + 1)   
        
    return df

In [61]:
df = normalize_df(data.iloc[:,2:])
df.head()

,returns,month spead,high off close,low off close,high of open,low off open,volume,Accounts Payable,Accounts Receivable,Add'l income/expense items,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
0,0.017750,-0.237354,-0.006863,0.006334,-0.008552,-0.008242,0.849448,0.006673,0.204119,-1.525988,...,-0.097916,0.371524,0.964126,-0.259769,-0.072938,-0.097857,0.744223,0.299171,1.112200,0.131225
1,0.040972,-0.241399,-0.000543,-0.005288,-0.002151,0.003596,0.680497,0.227325,-0.801420,-0.535363,...,-0.078480,0.945065,0.675777,-0.315340,-0.043053,-0.078421,0.145670,-0.205239,0.056006,1.244848
2,0.026131,-0.293996,0.000905,-0.005736,-0.000769,0.003984,0.271597,0.044816,-0.164156,0.816387,...,-0.127973,0.749414,0.494109,0.252412,-0.180401,-0.127914,0.064657,-0.306989,0.029089,1.096431
3,0.036966,-0.354683,-0.006072,0.000282,-0.002278,0.003452,-0.088412,0.551902,0.599153,0.644705,...,-0.132544,1.444840,0.953489,0.143504,-0.169978,-0.132551,1.626057,0.299171,0.035498,0.073823
4,-0.003353,1.441696,-0.000196,-0.000312,0.006551,0.007116,-0.583390,-0.261929,-0.037895,-0.128851,...,-0.206913,0.939303,0.198375,-0.378967,-0.178625,-0.206852,-0.396373,0.075051,0.794317,-0.562624


In [103]:
len(df.returns)

317

In [153]:
x_input = df.returns.to_numpy()
x = x_input.reshape(1, len(df.returns))
x.shape

(1, 317)

In [129]:
y = df.iloc[:,1:].to_numpy()
y.shape

(317, 81)

In [148]:

class Factors(nn.Module):
    def __init__(self, encoding_dim):
        super(Factors, self).__init__()
        #encoder 
        self.fc1 = nn.Linear(no_of_stocks, encoding_dim)
        
        #decoder 
        self.fc2 = nn.Linear(encoding_dim, no_of_metrics)
        
        #encoder  
        self.fc3 = nn.Linear(no_of_metrics_flat, encoding_dim)
        
        #decoder
        self.fc4 = nn.Linear(encoding_dim, no_of_metrics_flat)
        

    def forward(self, x, y):
       
        x = F.relu(self.fc1(x))
        
        x = (self.fc2(x))
        
        y = torch.flatten(y)
        
        y = F.relu(self.fc3(y))
        
        y = F.relu(self.fc4(y))
        
        y = y.view(no_of_metrics, no_of_stocks)
        x = x.view(1,no_of_metrics)
        
        w = torch.mm(x, y)
        
        return w

# initialize the NN
no_of_stocks = 317 #len(df.returns)
no_of_metrics = 81
no_of_metrics_flat = no_of_stocks * no_of_metrics  #metrics.shape[1] 
encoding_dim = 32

model = Factors(encoding_dim)
print(model)

Factors(
  (fc1): Linear(in_features=317, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=81, bias=True)
  (fc3): Linear(in_features=25677, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=25677, bias=True)
)


In [149]:
num_workers = 0 
x_batch = no_of_stocks
y_batch = no_of_stocks
x_loader = torch.utils.data.DataLoader(x, batch_size=x_batch, num_workers=num_workers)
y_loader = torch.utils.data.DataLoader(y, batch_size=y_batch, num_workers=num_workers)

In [150]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [151]:
# number of epochs to train the model
n_epochs = 100

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for x, y in zip(x_loader, y_loader):

        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(x.float(), y.float())
        # calculate the loss
        
        loss = criterion(outputs, x.float())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()
            
    # print avg training statistics 
    train_loss = train_loss/len(x_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.032876
Epoch: 2 	Training Loss: 1.403714
Epoch: 3 	Training Loss: 0.085510
Epoch: 4 	Training Loss: 0.006720
Epoch: 5 	Training Loss: 0.006307
Epoch: 6 	Training Loss: 0.005961
Epoch: 7 	Training Loss: 0.005667
Epoch: 8 	Training Loss: 0.005406
Epoch: 9 	Training Loss: 0.005176
Epoch: 10 	Training Loss: 0.004966
Epoch: 11 	Training Loss: 0.004771
Epoch: 12 	Training Loss: 0.004590
Epoch: 13 	Training Loss: 0.004420
Epoch: 14 	Training Loss: 0.004259
Epoch: 15 	Training Loss: 0.004106
Epoch: 16 	Training Loss: 0.003960
Epoch: 17 	Training Loss: 0.003819
Epoch: 18 	Training Loss: 0.003682
Epoch: 19 	Training Loss: 0.003550
Epoch: 20 	Training Loss: 0.003421
Epoch: 21 	Training Loss: 0.003295
Epoch: 22 	Training Loss: 0.003173
Epoch: 23 	Training Loss: 0.003053
Epoch: 24 	Training Loss: 0.002936
Epoch: 25 	Training Loss: 0.002822
Epoch: 26 	Training Loss: 0.002710
Epoch: 27 	Training Loss: 0.002602
Epoch: 28 	Training Loss: 0.002497
Epoch: 29 	Training Loss: 0.0

In [173]:
data.returns.head(5)

0   -0.021036
1    0.003728
2   -0.012099
3   -0.000545
4   -0.043540
Name: returns, dtype: float64

In [174]:
for x, y in zip(x_loader, y_loader):
    result = model(x.float(), y.float())
    
    unnormal_result = result * (data.returns.std() + 1) + data.returns.mean()
    
    print(unnormal_result.detach().numpy()[0,:5])

[-0.02137146  0.00388982 -0.01229126 -0.0003422  -0.0432324 ]
